# Скачивание и загрузка модели

In [11]:
import pymorphy2
import pandas as pd
import numpy as np
import gensim
from nltk.stem.snowball import SnowballStemmer
import inspect
from json import load, dump

import wget
import zipfile
import re

In [7]:
def download_model_rusvectores(model_id="65"):
    """"""
    if not os.path.exists("models"):
        os.mkdir("models")
    if os.path.exists(f"models/{model_id}"):
        print("Model is downloaded already")
    else:
        model_url = f'http://vectors.nlpl.eu/repository/11/{model_id}.zip'
        m = wget.download(model_url)
        with zipfile.ZipFile(f'{model_id}.zip', 'r') as zip_ref:
            unzip_path = "models/" + str(model_id)
            zip_ref.extractall(unzip_path)
    return f"models/{model_id}"

In [8]:
def load_model(model_path):
    model = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)
    return model

In [9]:
# добавляем название модели в путь
model_path = "models/65/model.bin"

# загрузка модели
model = load_model(model_path)

# Загрузка и обработка словаря

In [10]:
def get_gloss_df(file_name="RSL_class_list.txt", sep="\t"):
    df = pd.read_csv(file_name, sep="\t", names=['index', 'word'], index_col='index')
    return df

In [60]:
def clean_text(text):
    # Регулярное выражение, которое оставляет только буквы, цифры, пробелы и слэш
    pattern = r'[^a-zA-Zа-яА-Я0-9 /]'
    # Замена найденных совпадений на пустую строку
    clean_text = re.sub(pattern, '', text)
    return clean_text

In [80]:
def set_type(row):
    if row["composite"]:
        return "composite"
    elif row["synonym"]:
        return "synonym"
    else:
        return "single"


def glossary_typing(df):
    
    df['word'] = df['word'].apply(lambda x: clean_text(x))
    df['word'] = df['word'].apply(lambda x: x.replace("ё", "е"))

    df['composite'] = df['word'].apply(lambda x: True if len(x.split()) > 1 else False)
    df['synonym'] = df['word'].apply(lambda x: True if '/' in x else False)

    df = df[df['word'].str.split().apply(len) < 4]
    df = df[df['word'].str.split("/").apply(len) < 4]
    
    letters_to_remove = list('бгдежзйлмнопртфхцчьэыъю')
    # Создаем маску для удаления строк
    mask = df['word'].isin(letters_to_remove)
    # Удаляем строки
    df = df[~mask]
    
    df = df.query('word != ""')
    
    df = df.assign(word_type=df.apply(set_type, axis=1))
    return df

In [62]:
gloss = get_gloss_df()

In [81]:
gloss_typed = glossary_typing(gloss)

In [83]:
gloss_typed

,word,composite,synonym,word_type
index,,,,
0,1,False,False,single
1,10,False,False,single
2,2,False,False,single
3,2 часа,True,False,composite
4,20,False,False,single
...,...,...,...,...
1593,я,False,False,single
1594,являться,False,False,single
1595,язык,False,False,single


In [64]:
class WordAnalyzer:
    def __init__(self):
        """
        Инициализация объекта класса WordAnalyzer.
        """
        self.morph = self._init_morph()
        self.stemmer = SnowballStemmer("russian")

    def _init_morph(self):
        """
        Инициализация объекта MorphAnalyzer с передачей языка из параметров конструктора.
        """
        init_params = inspect.signature(pymorphy2.MorphAnalyzer.__init__).parameters
        lang_param = init_params.get('lang')
        lang = 'ru' if lang_param.default == inspect.Parameter.empty else lang_param.default
        return pymorphy2.MorphAnalyzer(lang=lang)

    def extract_normal_form(self, word: str) -> str:
        """
        Извлекает нормальную форму слова.

        Параметры:
            word (str): Слово, для которого нужно найти нормальную форму.

        Возвращает:
            str: Нормальная форма слова.
        """
        parsed_word = self.morph.parse(word)[0]
        return parsed_word.normal_form

    def _get_param_names(self):
        signature = inspect.signature(self.__init__)
        return [p.name for p in signature.parameters.values()]

    def stemming(self, word: str) -> str:
        """
        Выполняет стемминг слова.

        Параметры:
            word (str): Слово, которое нужно стеммить.

        Возвращает:
            str: Основа слова после стемминга.
        """
        return self.stemmer.stem(word)

    def levenshtein_distance(self, word1: str, word2: str) -> int:
        """
        Вычисляет расстояние Левенштейна между двумя словами.

        Параметры:
            word1 (str): Первое слово.
            word2 (str): Второе слово.

        Возвращает:
            int: Расстояние Левенштейна между словами.
        """
        len1, len2 = len(word1), len(word2)
        dp = [[0] * (len2 + 1) for _ in range(len1 + 1)]
        for i in range(len1 + 1):
            dp[i][0] = i
        for j in range(len2 + 1):
            dp[0][j] = j
        for i in range(1, len1 + 1):
            for j in range(1, len2 + 1):
                cost = 0 if word1[i - 1] == word2[j - 1] else 1
                dp[i][j] = min(dp[i-1][j] + 1, dp[i][j-1] + 1, dp[i-1][j-1] + cost)
        return dp[len1][len2]

    def compare_words(self, word1: str, word2: str, threshold_percent: float = 0.5) -> bool:
        """
        Сравнивает два слова на основе их нормальных форм, стеммов и расстояния Левенштейна.

        Параметры:
            word1 (str): Первое слово.
            word2 (str): Второе слово.
            threshold_percent (float): Процент от длины слова для определения порогового значения.
                                      По умолчанию 0.7.

        Возвращает:
            bool: True, если слова считаются однокоренными, False в противном случае.
        """
        normal_form1 = self.extract_normal_form(word1)
        normal_form2 = self.extract_normal_form(word2)
        stem1 = self.stemming(normal_form1)
        stem2 = self.stemming(normal_form2)
        distance_threshold = max(len(stem1), len(stem2)) * threshold_percent
        distance = self.levenshtein_distance(stem1, stem2)
        return distance <= distance_threshold

    def find_related_words(self, word: str, dictionary: list[str]) -> list[str]:
        """
        Находит однокоренные слова к поданному слову из заданного словаря.

        Параметры:
            word (str): Слово, для которого нужно найти однокоренные слова.
            dictionary (List[str]): Список слов из словаря.

        Возвращает:
            List[str]: Список однокоренных слов к поданному слову из словаря.
        """
        related_words = [dict_word for dict_word in dictionary if self.compare_words(word, dict_word)]
        return related_words

In [65]:
def search_similar_word_in_model(word: str) -> str:

    analyzer = WordAnalyzer()
    try:
        most_similar = model.most_similar(word)
        if len(most_similar) < 1:
            return None
        else:
            similar_words = {similar[0]: similar[1] for similar in most_similar}  # создание словаря похожих слов
            homonyms = analyzer.find_related_words(word, list(similar_words.keys()))  # Преобразуем представление ключей в список
            homonyms_dict = {word: similar_words[word] for word in similar_words.keys() if word in homonyms}
        if len(homonyms) < 1:               
            return list(similar_words.keys())[0]
        elif len(homonyms) == 1:
            list(homonyms_dict.keys())[0]
        else:
            best_distance = 0
            for key, val in homonyms_dict.items():  # Используем items() для итерации по ключам и значениям
                if val > best_distance:
                    best_distance = val
                    best_word = key
            return best_word                     
    except:
        return None

def get_vector(x, column):
    key = x[column]
    if key in model:
        return model[key]
    else: 
        return search_similar_word_in_model(key)

def add_synonyms_composit_columns(df):
    # Создаем столбцы для синонимов и для их векторных представлений
    for i in range(1, 4):
        df[f'synonym_{i}'] = df.apply(lambda x: x['word'].split('/')[i-1] if x['synonym'] and len(x['word'].split('/')) >= i else None, axis=1)
        df[f'synonym_{i}_vec'] = df.apply(lambda x: get_vector(x, f'synonym_{i}') if x['synonym'] else '', axis=1)
    
    # Создаем столбцы для составных слов и для их векторных представлений
    for i in range(1, 4):
        df[f'composite_{i}'] = df.apply(lambda x: x['word'].split()[i-1] if x['composite'] and  i <= len(x['word'].split()) < 4 else None, axis=1)
        df[f'composite_{i}_vec'] = df.apply(lambda x: get_vector(x, f'composite_{i}') if x['composite'] else None, axis=1)
    return df

In [84]:
gloss_typed_vec = add_synonyms_composit_columns(gloss_typed)

In [189]:
def get_vec_for_composite(df):
    df = df.query("word_type == 'composite'")
    composite_columns = [f'composite_{i}' for i in range(1, 4)]
    composite_vec_columns = [f'composite_{i}_vec' for i in range(1, 4)]
    df = df.fillna("")
    
    
    df["composite_3_vec"] = df["composite_3_vec"].apply(lambda x: [0.0] * 100 if x == "" else x)
    

    
    # Добавление новой колонки с суммой векторов
    df['vec'] = df.apply(lambda row: 
                                   np.round(np.array(row['composite_1_vec']) +
                                   np.array(row['composite_2_vec']) +
                                   np.array(row['composite_3_vec']), 6), axis=1)

    
    

    return df[["word", "vec"]]

In [190]:
gloss_vected_comp = get_vec_for_composite(gloss_typed_vec)

/var/folders/ns/04cy8gjs09v9bg9std1bl7l00000gn/T/ipykernel_79638/2128229662.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  df["composite_3_vec"] = df["composite_3_vec"].apply(lambda x: [0.0] * 100 if x == "" else x)


In [170]:
def select_synonym_vec(df):
    df1 = df.query("synonym_1 != ''")
    df1 = df1[["word", "synonym_1_vec"]]
    df1 = df1.rename(columns={"synonym_1_vec": "vec"})
    
    df2 = df.query("synonym_2 != ''")
    df2 = df2[["word", "synonym_2_vec"]]
    df2 = df2.rename(columns={"synonym_2_vec": "vec"})
    
    df3 = df.query("synonym_3 != ''")
    df3 = df3[["word", "synonym_3_vec"]]
    df3 = df3.rename(columns={"synonym_3_vec": "vec"})
    
    res = pd.concat([df1, df2, df3], ignore_index=True)
    return res

In [171]:
synonyms_from_dict = select_synonym_vec(gloss_vected)

In [218]:
def get_all_vec(df, synonyms, composite):
    df = df.query("word_type == 'single'")
    df['vec'] = df.apply(lambda x: get_vector(x, 'word'), axis=1)
    df = df[["word", "vec"]]
    
    res = pd.concat([df, synonyms, composite], ignore_index=True)
    res = res[~res["vec"].isna()]
    return res

In [219]:
gloss_vectors = get_all_vec(gloss_vected, synonyms_from_dict, gloss_vected_comp)

/var/folders/ns/04cy8gjs09v9bg9std1bl7l00000gn/T/ipykernel_79638/1409097487.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vec'] = df.apply(lambda x: get_vector(x, 'word'), axis=1)


In [ ]:
# Преобразование векторов в списки Python
    df['vec'] = df['vec'].apply(lambda x: x.tolist())
    
    # Создание словаря из DataFrame
    dict_vec = df.set_index('word')['vec'].to_dict()
    
    # Сохранение словаря в JSON файл
    with open(json_filename, 'w', encoding='utf-8') as json_file:
        json.dump(dict_vec, json_file, ensure_ascii=False, indent=2)

In [227]:
def dataframe_to_dict(df):
    df['vec'] = df['vec'].apply(list)
    dict_vec = df.set_index('word')['vec'].to_dict()
    return dict_vec
#     with open(json_filename, 'w', encoding='utf-8') as json_file:
#         dump(dict_vec, json_file, ensure_ascii=False, indent=2)

In [229]:
gloss_dict = dataframe_to_dict(gloss_vectors)

In [230]:
# сохранить в json

# Обработка входного файла

In [231]:
# обработать при помощи скрипта, которым обработан словарь

# Поиск похожего слова

In [232]:
# разбить на функции и привести в порядок

In [275]:
analyzer = WordAnalyzer()

In [6]:
def read_json_to_dict(json_filename: str) -> dict:

    with open(json_filename, 'r', encoding='utf-8') as json_file:
        data_dict = load(json_file)

    for key, value in data_dict.items():
        data_dict[key] = np.array(value, dtype=np.float32)
        
    return data_dict

In [1]:
def read_test_list(filename: str) -> list:

    with open(filename, encoding='UTF-8') as file:
        test = []
        for line in file.readlines():
            words = line.strip().split()
            for word in words:
                if word in digits.keys():
                    word = digits[word]
                else:
                    for old_char, new_char in to_replace.items():
                        word = word.replace(old_char, new_char)
                test.append(word)
    return test

In [278]:
def search_similar_word_in_model(word: str, analyzer) -> str:

    try:
        most_similar = model.most_similar(word)
        if len(most_similar) < 1:
            return None
        else:
            similar_words = {similar[0]: similar[1] for similar in most_similar}  # создание словаря похожих слов
            homonyms = analyzer.find_related_words(word, list(similar_words.keys()))  # Преобразуем представление ключей в список
            homonyms_dict = {word: similar_words[word] for word in similar_words.keys() if word in homonyms}
        if len(homonyms) < 1:               
            return list(similar_words.keys())[0]
        elif len(homonyms) == 1:
            return list(homonyms_dict.keys())[0]
        else:
            best_distance = 0
            for key, val in homonyms_dict.items():  # Используем items() для итерации по ключам и значениям
                if val > best_distance:
                    best_distance = val
                    best_word = key
            return best_word                     
    except KeyError:
        return None

In [270]:
from typing import Union

In [279]:
def prepare_test_dict(filename: str, analyzer) -> Union[dict, list]:
    not_found = []
    test = read_test_list(filename)
    test_dict = {}
    for word in test:
        if word in model:
            test_dict[word] = model[word] 
        else:
            lemma = analyzer.extract_normal_form(word)
            try:
                vector = model[lemma]  # Пытаемся найти вектор для слова в модели
                test_dict[word] = vector
            except KeyError:
                similar_word = search_similar_word_in_model(word)
                if similar_word:
                    test_dict[word] = similar_word
                else:
                    not_found.append(word)
    return test_dict, not_found

In [273]:
input_type = 'filename' # можно подавать на вход функции имя файла 'filename' со словами, список слов 'lst', слово 'word', текст 'str', каждый тип данных на входе будет обработан в соответсвии с типом
test_filename = 'test.txt'
json_dict_name = 'gloss_dict.json'
not_found = []
gloss_dict = read_json_to_dict(json_dict_name)
text_in_gloss = []
trashhold = 0.3

if input_type == 'filename':
    test_dict, not_found = prepare_test_dict(test_filename, analyzer)

    for word in test_dict:
        if word in gloss_dict:
            text_in_gloss.append(word)
        else:
            similarity_lst = []
            for gloss, vec in gloss_dict.items():
                cos_sim = np.dot(test_dict[word], gloss_dict[gloss])/(np.linalg.norm(test_dict[word]) * np.linalg.norm(gloss_dict[gloss]))
                if len(similarity_lst) < 1:
                    similarity_lst.append((gloss, cos_sim))
                    top_ten_distance = cos_sim
                if len(similarity_lst) < 11:
                    similarity_lst.append((gloss, cos_sim))
                    if cos_sim < top_ten_distance:
                        top_ten_distance = cos_sim
                else:
                    if cos_sim > top_ten_distance:
                        similarity_lst = [tup for tup in similarity_lst if tup[1] != top_ten_distance]
                        similarity_lst.append((gloss, cos_sim))
                        similarity_lst = sorted(similarity_lst, key=lambda x: x[1], reverse=True)
                        top_ten_distance = similarity_lst[9][1]
            
            similarity_lst = sorted(similarity_lst, key=lambda x: x[1], reverse=True)
            print(word, similarity_lst)
            similar_words = {similar[0]: similar[1] for similar in similarity_lst}  # создание словаря похожих слов
            homonyms = analyzer.find_related_words(word, list(similar_words.keys()))  # Преобразуем представление ключей в список
            homonyms_dict = {word: similar_words[word] for word in similar_words.keys() if word in homonyms}
            if len(homonyms) < 1:               
                text_in_gloss.append(similarity_lst[0][0])
            elif len(homonyms) == 1:
                text_in_gloss.append(list(homonyms_dict.keys())[0])
            else:
                best_distance = 0
                for key, val in homonyms_dict.items():  # Используем items() для итерации по ключам и значениям
                    if val > best_distance:
                        best_distance = val
                        best_word = key
                text_in_gloss.append(best_word)
    
print(list(test_dict.keys()))
print(text_in_gloss)


# независимо от типа входных данных функция всегда возвращает список слов 'str' из словаря глоссов и список слов, которые не нашли
# На выходе из нашего скрипта, мы предоставляем спискок слов из словаря без векторных представлений а также файл gloss_dict.json для дальнейшей работы с одним и тем же предобработанным словарем глоссов,
# который содежит слово и его векторное представление мз модели word2vec, а также список слов, которых нет в модели и наш скрипт их не обработал

чем [('пот', 0.24484825), ('попадать', 0.24069232), ('себе', 0.23570879), ('второй', 0.235332), ('жадно есть', 0.23294939), ('жадноесть', 0.23294939), ('деньга', 0.23260924), ('тридцать', 0.2313614), ('возникать', 0.23004192), ('упасть', 0.22939818), ('безопасность', 0.10240044)]
невоз [('практически', 0.42358407), ('так себе', 0.40360782), ('таксебе', 0.40360782), ('переходить', 0.3924577), ('но', 0.38199118), ('только', 0.37925637), ('аллергия на еду', 0.3787159), ('аллергиянаеду', 0.3787159), ('вообще', 0.37783754), ('получаться', 0.3726462), ('4', 0.120081045)]
завтрак [('закуска перед едой', 0.3274034), ('закускапередедой', 0.3274034), ('еду', 0.32398418), ('хлеб', 0.3227446), ('жадно есть', 0.3163149), ('жадноесть', 0.3163149), ('пищи', 0.30907604), ('носорог', 0.30630806), ('вкусный', 0.30491206), ('ароматный', 0.3027006), ('активный', 0.12142241)]
трое [('дельфин', 0.15827794), ('оседать', 0.15710601), ('вагон', 0.15534014), ('автобус', 0.1514308), ('пот', 0.1504958), ('проект'

In [222]:
gloss_dict.keys()

dict_keys(['1', '10', '2', '2 часа', '20', '3', '4', '5', '6', '6 часов', '7', '8', '9', 'нет жеста', 'а', 'август', 'автобус', 'агрессивный', 'агрессия', 'адаптивное поведение', 'адаптивность', 'адаптивный', 'адрес/улица', 'аккуратный', 'активный', 'актер', 'акула', 'аллергия на еду', 'аллигатор', 'амбициозный', 'английский', 'аниматор', 'аппетит', 'аренда', 'армия', 'ароматный', 'аэропорт', 'бабочка', 'багаж', 'база', 'банк', 'банкет', 'башенные часы', 'бегемот', 'бежать', 'бежевый', 'без', 'безопасность', 'безразличие', 'безумный', 'белок', 'белый', 'берег', 'беспокоить', 'беспокоиться', 'беспокойный', 'беспокойство', 'бизнес', 'билет', 'благоговение', 'благодарность', 'благополучие', 'бледный', 'близкий', 'бог', 'бой', 'более', 'болеть', 'большинство', 'большой', 'борода', 'борьба', 'бояться', 'брат', 'брать', 'бронь', 'бросать', 'брюзжать', 'будто', 'будущий', 'бумага', 'бутылка', 'бы', 'бывать', 'бывший', 'бык', 'быстрый', 'быть', 'быть потрясенным', 'бюро', 'в', 'в восемь пятнад

In [4]:
# добавляем название модели в путь
model_path = "models/65/model.bin"

# загрузка модели
model = load_model(model_path)

In [9]:
json_dict_name = 'gloss_dict.json'
gloss_dict = read_json_to_dict(json_dict_name)